## Auto tag the tickets with right priorities and right departments so that reassigning and related delay can be reduced.

### Connecting to DataBase


In [2]:
from sqlalchemy import create_engine
import pandas as pd

db_host = '18.136.56.185'
username ='dm_team'
pwd='dm_team123#'
db_name='project_itsm'

#Connecting to SQL DB
conn =create_engine('mysql+pymysql://'+username+':'+pwd+'@'+db_host+'/'+db_name)
conn.table_names()

['dataset_list']

### Loading Data

In [3]:
query ='select * from dataset_list '
df =pd.read_sql(query,conn )
print(df.shape)
df.head(2)

(46606, 25)


,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,Closed,4,4,4,0.601292279,...,,04-11-2013 13:50,04-11-2013 13:51,"3,87,16,91,111",Other,1,SD0000007,2,,
1,WBA000124,application,Web Based Application,WBS000088,IM0000005,Closed,3,3,3,0.415049969,...,02-12-2013 12:31,02-12-2013 12:36,02-12-2013 12:36,"4,35,47,86,389",Software,1,SD0000011,1,,


In [4]:
df.columns

Index(['CI_Name', 'CI_Cat', 'CI_Subcat', 'WBS', 'Incident_ID', 'Status',
       'Impact', 'Urgency', 'Priority', 'number_cnt', 'Category', 'KB_number',
       'Alert_Status', 'No_of_Reassignments', 'Open_Time', 'Reopen_Time',
       'Resolved_Time', 'Close_Time', 'Handle_Time_hrs', 'Closure_Code',
       'No_of_Related_Interactions', 'Related_Interaction',
       'No_of_Related_Incidents', 'No_of_Related_Changes', 'Related_Change'],
      dtype='object')

In [5]:
df['CI_Cat_New']=df['CI_Cat']
df.loc[:,['Open_Time','Close_Time','Handle_Time_hrs']]

,Open_Time,Close_Time,Handle_Time_hrs
0,05-02-2012 13:32,04-11-2013 13:51,"3,87,16,91,111"
1,12-03-2012 15:44,02-12-2013 12:36,"4,35,47,86,389"
2,29-03-2012 12:36,13-01-2014 15:13,"4,84,31,19,444"
3,17-07-2012 11:49,14-11-2013 09:31,"4,32,18,33,333"
4,10-08-2012 11:01,08-11-2013 13:55,"3,38,39,03,333"
...,...,...,...
46601,31-03-2014 16:23,31-03-2014 16:29,"0,095"
46602,31-03-2014 15:03,31-03-2014 15:29,"0,428333333"
46603,31-03-2014 15:28,31-03-2014 15:32,"0,071666667"
46604,31-03-2014 15:35,31-03-2014 15:42,"0,116944444"


## EDA

In [6]:
from collections import Counter
import numpy as np

# CI_Cat filed cleaning 
Counter(df.CI_Cat)
df.CI_Cat.isnull().sum()

#Replacing empty srting values of 111 records of CI_Cat column n dropping the same

df['CI_Cat'] = np.where((df.CI_Cat == ''),np.nan,df.CI_Cat)
Counter(df['CI_Cat'])

df.dropna(inplace=True)

df.isnull().sum()
Counter(df['CI_Cat'])

#Replacing empty srting values of 111 records of CI_Cat column n dropping the same

df['CI_Subcat'] = np.where((df.CI_Subcat == ''),np.nan,df.CI_Subcat)
Counter(df['CI_Subcat'])

df.dropna(inplace=True)

df.isnull().sum()
Counter(df['CI_Subcat'])


Counter({'Web Based Application': 15311,
         'Desktop Application': 3876,
         'Server Based Application': 18811,
         'SAP': 1199,
         'Client Based Application': 492,
         'Citrix': 767,
         'Standard Application': 85,
         'Windows Server': 153,
         'Laptop': 1921,
         'Linux Server': 55,
         'Monitor': 212,
         'Automation Software': 55,
         'SAN': 549,
         'Banking Device': 916,
         'Desktop': 423,
         'Database': 211,
         'Oracle Server': 3,
         'Keyboard': 27,
         'Printer': 88,
         'Exchange': 130,
         'System Software': 272,
         'VDI': 9,
         'Encryption': 45,
         'Omgeving': 86,
         'MigratieDummy': 60,
         'Scanner': 64,
         'Controller': 146,
         'DataCenterEquipment': 277,
         'KVM Switches': 32,
         'Switch': 28,
         'Database Software': 4,
         'Network Component': 32,
         'Unix Server': 7,
         'Lines': 15,
      

### Cleaning columns : Impact,Urgency,Priority

In [7]:
#-----------Impact column have junk values 'NS' so it has to be replced by valid value
df.loc[(df.Impact=='NS'),'Impact']= df.Urgency
Counter(df.Impact)

#-----------Urgency column have '5 - Very Low': rename it to 5
df.loc[(df.Urgency=='5 - Very Low')] = '5'

#----------Priority column has NA values replce with urgency values
Counter(df.Priority)
df.loc[(df.Priority == 'NA'),'Priority'] =df.Urgency
df.loc[(df.Priority==5),'Priority'] ='5'

Counter(df.Priority)

Counter({'4': 22721, '3': 6655, '5': 16427, '2': 689, '1': 3})

### Cleaning WBS column

In [8]:
df.columns
Counter(df.WBS)

df.loc[(df.WBS == '5'),'WBS'] = np.nan


df.isnull().sum()
df.loc[:,'WBS'] = df.loc[:,'WBS'].ffill()
#df.WBS = df['WBS'].ffill()
df.isnull().sum()

Counter(df.WBS)

Counter({'WBS000162': 495,
         'WBS000088': 619,
         'WBS000092': 784,
         'WBS000055': 115,
         'WBS000090': 128,
         'WBS000073': 13342,
         'WBS000066': 154,
         'WBS000071': 21,
         'WBS000263': 2283,
         'WBS000072': 2197,
         'WBS000054': 165,
         'WBS000271': 1186,
         'WBS000170': 70,
         'WBS000014': 219,
         'WBS000098': 208,
         'WBS000043': 232,
         'WBS000086': 80,
         'WBS000027': 166,
         'WBS000300': 43,
         'WBS000223': 803,
         'WBS000311': 16,
         'WBS000142': 2,
         'WBS000109': 1,
         'WBS000153': 146,
         'WBS000102': 185,
         'WBS000152': 1076,
         'WBS000129': 2,
         'WBS000251': 42,
         'WBS000115': 17,
         'WBS000168': 72,
         'WBS000136': 18,
         'WBS000314': 3,
         'WBS000187': 353,
         'WBS000135': 54,
         'WBS000217': 70,
         'WBS000091': 2500,
         'WBS000296': 582,
         'WBS

In [14]:
df['No_of_Reassignments'] = np.where((df.No_of_Reassignments == ''),'1',df.No_of_Reassignments)
Counter(df.No_of_Reassignments)



Counter({'26': 7,
         '33': 3,
         '3': 2185,
         '13': 61,
         '2': 5369,
         '4': 1601,
         '5': 721,
         '6': 622,
         '8': 246,
         '17': 27,
         '1': 7252,
         '7': 329,
         '12': 58,
         '0': 27395,
         '11': 101,
         '9': 170,
         '25': 10,
         '30': 3,
         '15': 43,
         '37': 1,
         '32': 3,
         '22': 10,
         '10': 146,
         '21': 11,
         '19': 10,
         '14': 46,
         '46': 1,
         '18': 14,
         '16': 25,
         '42': 1,
         '23': 5,
         '39': 1,
         '20': 7,
         '45': 1,
         '38': 1,
         '24': 3,
         '34': 1,
         '29': 1,
         '27': 2,
         '31': 1,
         '36': 1})

### Label ENcoding of columns CI_Cat ,CI_SubCat and Catogery

In [15]:
from sklearn.preprocessing import LabelEncoder

encoder= LabelEncoder()

df.CI_Cat = encoder.fit_transform(df['CI_Cat'])
df.CI_Cat 

df.CI_Subcat =encoder.fit_transform(df['CI_Subcat'])
df.CI_Subcat

Counter(df.Category)
df.Category = encoder.fit_transform(df.Category)
df.Category

df.WBS = encoder.fit_transform(df.WBS)
df.WBS

0        136
1         70
2         74
3         70
4         70
        ... 
46601     59
46602     59
46603     73
46604     59
46605    216
Name: WBS, Length: 46495, dtype: int32

### Define X & Y   , split the data

In [16]:
#Since Priority can be determined by considering the Impact and Urgency
X=df.loc[:,['CI_Subcat','Category','Priority','WBS']]
Y=df.CI_Cat


In [17]:
from sklearn.model_selection import train_test_split

#Dividing data into train n test dataset
X_train ,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=10)

### Define Model :  RandomForestClassifier

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score

model_rf = RandomForestClassifier(max_depth= 15, n_estimators= 1000,random_state=10)
model_rf.fit(X_train,Y_train)
Y_predict= model_rf.predict(X_test)

accuracy_score(Y_test,Y_predict)

0.8998494515735895

In [331]:
df.loc[:,['No_of_Related_Changes','No_of_Related_Interactions']]
Counter(df.No_of_Related_Interactions)

Counter({'1': 42962,
         '2': 2723,
         '3': 427,
         '14': 2,
         '7': 12,
         '4': 123,
         '5': 31,
         '370': 1,
         '9': 5,
         '11': 2,
         '54': 1,
         '': 114,
         '288': 1,
         '34': 2,
         '44': 2,
         '39': 2,
         '12': 6,
         '42': 3,
         '6': 16,
         '8': 6,
         '28': 2,
         '29': 2,
         '13': 4,
         '18': 2,
         '20': 5,
         '15': 3,
         '31': 1,
         '88': 1,
         '30': 1,
         '41': 1,
         '74': 1,
         '16': 2,
         '24': 1,
         '17': 2,
         '57': 1,
         '33': 2,
         '10': 3,
         '118': 1,
         '45': 2,
         '55': 1,
         '40': 1,
         '19': 3,
         '22': 3,
         '23': 2,
         '37': 1,
         '26': 2,
         '43': 1,
         '25': 1,
         '27': 1,
         '21': 1})

In [36]:
df['No_of_Related_Changes']=np.where((df['No_of_Related_Changes']==''), '1',df['No_of_Related_Changes'])

df.loc[:,['CI_Cat','No_of_Related_Changes']]
print(Counter(df.No_of_Reassignments))

print(Counter(df.Priority))

Counter({'0': 27395, '1': 7252, '2': 5369, '3': 2185, '4': 1601, '5': 721, '6': 622, '7': 329, '8': 246, '9': 170, '10': 146, '11': 101, '13': 61, '12': 58, '14': 46, '15': 43, '17': 27, '16': 25, '18': 14, '21': 11, '25': 10, '22': 10, '19': 10, '26': 7, '20': 7, '23': 5, '33': 3, '30': 3, '32': 3, '24': 3, '27': 2, '37': 1, '46': 1, '42': 1, '39': 1, '45': 1, '38': 1, '34': 1, '29': 1, '31': 1, '36': 1})
Counter({'4': 22721, '5': 16427, '3': 6655, '2': 689, '1': 3})


#### #Retrivinig the department whose 'No_of_Reassignments > 1'

In [37]:
import datetime

df.loc[:,['No_of_Related_Interactions', 'No_of_Related_Changes', 'Related_Change']]

df1 = df.loc[df['No_of_Reassignments'] > '2']
df1.loc[:,['CI_Cat','CI_Cat_New','No_of_Reassignments','Priority']]
Counter(df1.Priority)

#,'Open_Time','Close_Time','Handle_Time_hrs'

Counter({'4': 2702, '3': 626, '5': 2591, '2': 29})

In [46]:
import warnings
warnings.filterwarnings("ignore")


df1['Open_Time'] =pd.to_datetime(df1['Open_Time'] ,errors='coerce')
df1['Open_Time'] = df1['Open_Time'].dt.date

df1['Close_Time'] =pd.to_datetime(df1['Close_Time'] ,errors='coerce')
df1['Close_Time'] = df1['Close_Time'].dt.date

df1.loc[:,['Open_Time','Close_Time']] 

df1['Difference'] = df1['Close_Time']- df1['Open_Time']
df1.loc[:,['CI_Cat','CI_Cat_New','No_of_Reassignments','Priority','Close_Time','Open_Time','Difference']]

# From the below data it can be concluded that Incidents which belong to 'Application' and 'Subapplication' have been 
# reassigned to 2 or more than 2 departments ,and have associated with low priority i,e 3,4,5

Counter(df1.CI_Cat_New)
df1.loc[:,['CI_Cat','CI_Cat_New','No_of_Reassignments','Priority','Close_Time','Open_Time','Difference']]


,CI_Cat,CI_Cat_New,No_of_Reassignments,Priority,Close_Time,Open_Time,Difference
0,12,subapplication,26,4,2013-04-11,2012-05-02,344 days
1,2,application,33,3,2013-02-12,2012-12-03,71 days
2,2,application,3,3,2014-01-13,2012-03-29,655 days
5,2,application,4,4,2013-08-11,2012-10-08,307 days
7,2,application,5,4,2013-08-11,2012-08-22,354 days
...,...,...,...,...,...,...,...
46268,12,subapplication,4,5,2014-03-31,2014-03-28,3 days
46362,2,application,3,2,2014-03-28,2014-03-28,0 days
46494,2,application,4,5,2014-03-31,2014-03-31,0 days
46558,2,application,5,5,2014-03-31,2014-03-31,0 days


In [45]:
df1.head(4)

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change,CI_Cat_New,Difference
0,SUB000508,12,58,136,IM0000004,Closed,4,4,4,0.601292279,...,2013-04-11,"3,87,16,91,111",Other,1,SD0000007,2,1,,subapplication,344 days
1,WBA000124,2,58,70,IM0000005,Closed,3,3,3,0.415049969,...,2013-02-12,"4,35,47,86,389",Software,1,SD0000011,1,1,,application,71 days
2,DTA000024,2,11,74,IM0000006,Closed,3,3,3,0.517551335,...,2014-01-13,"4,84,31,19,444",No error - works as designed,1,SD0000017,,1,,application,655 days
5,WBA000124,2,58,70,IM0000013,Closed,4,4,4,0.006675648,...,2013-08-11,"3,38,34,36,944",Other,1,SD0000031,,1,,application,307 days
